In [7]:
!pip install sqlalchemy_utils==0.36.8
!pip install cryptography==3.3.2 
!pip install SQLAlchemy==1.3.22 --upgrade
!pip install pandas==0.25.2 
!pip install psycopg2-binary

Reason for being yanked: Wrong required python
Processing /home/mosaic-ai/.cache/pip/wheels/68/31/b6/a96bf6868f42753696d647846c9a0f8e51bd99295790d07660/SQLAlchemy_Utils-0.36.8-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5e/9a/00fcc92faa439c869f89b3bd3a6bcd2512f1430d2631484ee2ada1d7d4ac/SQLAlchemy-1.4.31-cp36-cp36m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d9/5a/e7c31adbe875f2abbb91bd84cf2dc52d792b5a01506781dbcf25c91daf11/six-1.16.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9e/dc/0b1aeaa008144d3d503113aa4c5c3c46311b9942f77c1ae337d04fb703dc/greenlet-1.1.2-cp36-cp36m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/a0/a1/b153a0a4caf7a7e3f15c2cd56c7702e2cf3d89b1b359d1f1c5e59d68f4ce/importlib_metadata-4.8.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/package

You should consider upgrading via the 'pip install --upgrade pip' command.
  Using cached https://files.pythonhosted.org/packages/6e/cf/26c04273303a5ea33d33a41c7b051ec12df8d899a2d588c5209b84552957/SQLAlchemy-1.3.22-cp36-cp36m-manylinux2010_x86_64.whl
You should consider upgrading via the 'pip install --upgrade pip' command.
  Using cached https://files.pythonhosted.org/packages/86/12/08b092f6fc9e4c2552e37add0861d0e0e0d743f78f1318973caad970b3fc/pandas-0.25.2-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d3/e3/d9f046b5d1c94a3aeab15f1f867aa414f8ee9d196fae6865f1d6a0ee1a0b/pytz-2021.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/14/32/d3fa649ad7ec0b82737b92fefd3c4dd376b0bb23730715124569f38f3a08/numpy-1.19.5-cp36-cp36m-manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/36/7a/87837f39d0296e723bb9b62bbb257d0355c7f6128853c78955f57342a56d/python_dateutil-2.8.2-py2.py3-none-any.whl
  Using cached

In [9]:
!pip install SQLAlchemy==1.3.22 --upgrade

  Using cached https://files.pythonhosted.org/packages/6e/cf/26c04273303a5ea33d33a41c7b051ec12df8d899a2d588c5209b84552957/SQLAlchemy-1.3.22-cp36-cp36m-manylinux2010_x86_64.whl
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
pip list | grep SQLAlchemy

SQLAlchemy                        1.4.31    
SQLAlchemy-Utils                  0.36.8    
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import logging
import requests
import sqlalchemy as db
import os
from uuid import uuid4
import pandas as pd

LOGGER = logging.getLogger("test")

/opt/conda/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
import six
from cryptography.hazmat.backends import default_backend
from sqlalchemy_utils.types.encrypted.encrypted_type import FernetEngine
from cryptography.hazmat.primitives import hashes
from cryptography.fernet import Fernet
import sqlalchemy as db
import pandas as pd

def secret_key(key):
    if isinstance(key, six.string_types):
        key = key.encode()
    digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
    digest.update(key)
    engine_key = digest.finalize()
    f = FernetEngine()
    f._initialize_engine(engine_key)
    
    
    return f.secret_key

def encrypt(message):
    """
    :param message: message To be encrypted
    :return:
    """
    return f.encrypt(bytes(message, "utf-8")).decode("utf-8")


def decrypt(enctypted_message):
    """
    Message to be decrypted
    :param enctypted_message:
    :return:
    """
    return f.decrypt(bytes(enctypted_message, "utf-8")).decode("utf-8")
class VaultEncrypter:
    """
    Class implementing Vault Backend APIs
    """
    def __init__(self, prefix_path):
        """
        :param prefix_path: Prefix path of kv store
        :param base_url: vault url
        :param token:
        """
        self._prefix_path = prefix_path
        self._vault_url = os.environ["VAULT_URL"]
        self._headers = {"X-Vault-Token": os.environ["VAULT_TOKEN"]}

    @property
    def url(self) -> str:
        """
        :return: URL
        """
        return f"{self._vault_url}{self._prefix_path}"

    def retrieve(self, key, key_error="raise"):
        """
        :param key_error: raise or ignore
        :param key:
        :return:
        """
        if key is None:
            return None

        LOGGER.info("Entering retrieve, key %s", key)
        url = f"{self.url}/{key}"
        LOGGER.info("url... %s", url)

        resp = requests.get(url=url, headers=self._headers)
        try:
            resp.raise_for_status()
            resp = resp.json()
            value = resp["data"]["value"]

        except requests.exceptions.HTTPError as ex:
            if resp.status_code != 404 or key_error != "ignore":
                LOGGER.exception(ex)
                raise KeyError("Key Not Present in Vault")
            LOGGER.info("Key Not Present. Return None")
            value = None
        LOGGER.info("Exiting retrieve")
        return value

    def store(self, key=None, value=None):
        """
        :param key:
        :param value:
        :return:
        """
        if value is None:
            return None
        if key is None:
            key = str(uuid4())
        LOGGER.info("Entering store, key %s, value %s", key, value)
        url = f"{self.url}/{key}"
        payload = {"value": value}
        LOGGER.info("url... %s", url)
        LOGGER.info("payload... %s", payload)
        result = requests.post(url=url, json=payload, headers=self._headers)
        result.raise_for_status()
        LOGGER.info("Exiting store")
        return key

    def delete(self, key):
        """
        :param key:
        :param value:
        :return:
        """
        if key is None:
            return
        LOGGER.info("Entering Delete, key %s", key)
        url = f"{self.url}/{key}"
        LOGGER.info("url... %s", url)
        result = requests.delete(url=url, headers=self._headers)
        try:
            result.raise_for_status()
        except Exception as ex: # pylint: disable=broad-except
            LOGGER.exception(ex)
        LOGGER.info("Exiting delete")

In [3]:

ENC_PASSWORD = "NWdUMqomsoRPlROH_qFqb9vLeXlukVrXwwIll_8WhxE=" # same as used in k8 secret
os.environ["VAULT_TOKEN"]="s.qw3fpdglwSkCvSCsIjtMZIEi" # GA
os.environ["VAULT_URL"] = "http://release-name-vault.vault.svc.cluster.local:8200/v1/kv/data/"
# CONN_STRING_QA = "mysql+mysqlconnector://maxiq@mosaic-k8s-qa:QA_mysql_21@mosaic-k8s-qa.mysql.database.azure.com/ai_logistics"
CONN_STRING_GA ="mysql+mysqlconnector://root:M@xiq123@18.234.226.186:3306/ai_logistics?ssl_verify_cert=false&ssl_ca=false&ssl_disabled=true"
fernet_key = secret_key(ENC_PASSWORD)
f = Fernet(fernet_key)

engine = db.create_engine(CONN_STRING_GA)
# conn = engine.connect()
# conn.close()

In [6]:
conn = engine.connect()

TABLE = "nb_git_repository"
changes = []
errors = []
update_stmts=[]
resp = conn.execute(f"SELECT password, repo_id from {TABLE} where created_on is NULL")
vault = VaultEncrypter(TABLE)
for res in resp:
    old = res[0]
    repo_id = res[1]
    try:
        new = encrypt(old)
#         new = vault.store(value=decrypted)
        
        changes.append({"old":old,"new":new, "repo_id": repo_id})
        # Uncomment below line once after verifying, dataframe and run again.
        conn.execute(f"UPDATE nb_git_repository SET password='{new}' where repo_id='{res[1]}'")
        update_stmts.append(f"UPDATE nb_git_repository SET password='{new}' where repo_id='{res[1]}'")
    except Exception:
        print(f"error - {repo_id}")
        errors.append({"repo_id":repo_id})
        

conn.close()
df = pd.DataFrame(changes)
df.to_csv("migration-qa.csv")
df

updates = pd.DataFrame(update_stmts)
updates.to_csv("updates-ga.csv")


In [ ]:
updates[0] = updates[0]+";"

In [ ]:
updates.to_csv("updates-ga-2.csv", index=False)


In [ ]:
updates.shape

In [4]:
decrypt("gAAAAABhKMQGoyhk-Mklbqmlls1hO322Ex2QYABhtoTZBfQsGDJHdjNUMJddcfGjXfivVGn_g5GmwnJ5vMGmFdmNaKcXuaZeu-f9NDuVtgfhCjoiZjF-NDo=")

'Tz38yiz_x2ktN4esbvyp'